This script describes how we processed the SHARE-seq data of the hair follicle development study, preparing them for training and downstream analysis.

**Note:**

Before running the following scripts, download processed data of the hair follicle development study from the scglue package (http://download.gao-lab.org/GLUE/dataset/Ma-2020-RNA.h5ad and http://download.gao-lab.org/GLUE/dataset/Ma-2020-ATAC.h5ad), and convert the anndata files into ".h5seurat" format following instructions from the tutorial (https://mojaveazure.github.io/seurat-disk/articles/convert-anndata.html).

In [ ]:
library(Matrix)
library(Signac)
library(Seurat)
library(SeuratDisk)
library(EnsDb.Mmusculus.v79)
library(dplyr)
library(ggplot2)
#library(BSgenome.Hsapiens.UCSC.hg38)

set.seed(42)
setwd("/nfs/public/xixi/scRegulate/SHARE-seq")

In [ ]:
atac <- LoadH5Seurat("Ma-2020-ATAC.h5seurat")
atac

In [ ]:
rna <- LoadH5Seurat("Ma-2020-RNA.2.h5seurat", meta.data = FALSE)
rna

In [ ]:
# Create Seurat object
skin <- CreateSeuratObject(counts = rna@assays$RNA@counts)
skin[["percent.mt"]] <- PercentageFeatureSet(skin, pattern = "^MT-")

In [ ]:
# Now add in the ATAC-seq data
# we'll only use peaks in standard chromosomes
grange.counts <- StringToGRanges(rownames(atac), sep = c(":", "-"))
grange.use <- seqnames(grange.counts) %in% standardChromosomes(grange.counts)
atac_counts <- atac@assays$RNA@counts[as.vector(grange.use), ]
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)
seqlevelsStyle(annotations) <- 'UCSC'
genome(annotations) <- "mm10"

frag.file <- "GSM4156597_skin.late.anagen.atac.fragments.bed.gz"
chrom_assay <- CreateChromatinAssay(
   counts = atac_counts,
   sep = c(":", "-"),
   genome = 'mm10',
   min.cells = 10,
   annotation = annotations
 )
skin[["ATAC"]] <- chrom_assay

In [ ]:
VlnPlot(skin, features = c("nCount_ATAC", "nCount_RNA","percent.mt"), ncol = 3,
  log = TRUE, pt.size = 0) + NoLegend()

In [ ]:
skin <- subset(
  x = skin,
  subset = nCount_ATAC < 3e4 &
    nCount_ATAC > 500 &
    nCount_RNA < 8000 &
    nCount_RNA > 100 &
    percent.mt < 20
)
skin

In [ ]:
# RNA analysis
DefaultAssay(skin) <- "RNA"

skin <- FindVariableFeatures(skin, nfeatures = 3000)
skin <- NormalizeData(skin)
skin <- ScaleData(skin)

In [ ]:
skin <- RunPCA(skin)

In [ ]:
skin <- RunUMAP(skin, dims = 1:30, reduction.name = "umap.rna")
skin <- FindNeighbors(skin, dims = 1:30)
skin <- FindClusters(skin, resolution = 0.5, algorithm = 3)

In [ ]:
# ATAC analysis
# We exclude the first dimension as this is typically correlated with sequencing depth
DefaultAssay(skin) <- "ATAC"
skin <- FindTopFeatures(skin, min.cutoff = 10)
skin <- RunTFIDF(skin)
skin <- RunSVD(skin)
skin <- RunUMAP(skin, reduction = 'lsi', dims = 2:30, reduction.name = "umap.atac", reduction.key = "atacUMAP_")

In [ ]:
skin <- FindNeighbors(skin, reduction = 'lsi', dims = 2:30)

In [ ]:
skin <- FindClusters(skin, resolution = 1, algorithm = 3)

In [ ]:
skin <- FindMultiModalNeighbors(skin, reduction.list = list("pca", "lsi"), dims.list = list(1:30, 2:30))
skin <- RunUMAP(skin, nn.name = "weighted.nn", reduction.name = "wnn.umap", reduction.key = "wnnUMAP_")

In [ ]:
skin <- FindClusters(skin, graph.name = "wsnn", resolution = 1.5)

In [ ]:
skin$celltype <- atac$cell_type

In [ ]:
options(repr.plot.width=14, repr.plot.height=6)

In [ ]:
p1 <- DimPlot(skin, reduction = "umap.rna", label = TRUE, label.size = 2.5, repel = TRUE) + ggtitle("RNA")
p2 <- DimPlot(skin, reduction = "umap.atac", label = TRUE, label.size = 2.5, repel = TRUE) + ggtitle("ATAC")
p3 <- DimPlot(skin, reduction = "wnn.umap", label = TRUE, label.size = 2.5, repel = TRUE) + ggtitle("WNN")
p1 + p2 + p3 & NoLegend() & theme(plot.title = element_text(hjust = 0.5))

In [ ]:
p1 <- DimPlot(skin, reduction = "umap.rna", group.by = "celltype", label = TRUE, label.size = 2.5, repel = TRUE) + ggtitle("RNA")
p2 <- DimPlot(skin, reduction = "umap.atac", group.by = "celltype", label = TRUE, label.size = 2.5, repel = TRUE) + ggtitle("ATAC")
p3 <- DimPlot(skin, reduction = "wnn.umap", group.by = "celltype", label = TRUE, label.size = 2.5, repel = TRUE) + ggtitle("WNN")
p1 + p2 + p3 & NoLegend() & theme(plot.title = element_text(hjust = 0.5))

In [ ]:
saveRDS(skin, "skin.rna.atac.seuratobj.rds")

# Extract TAC, Medulla, and Hair Shaft-Cuticle/Cortex cells

In [ ]:
skin <- readRDS("skin.rna.atac.seuratobj.rds")
skin

In [ ]:
skin_realsub <- subset(skin, subset = seurat_clusters %in% c(4,7,8,10,17,22))
#skin_realsub <- skin[,  skin$seurat_clusters %in% c(4,7,8,10,17,22)]

In [ ]:
DefaultAssay(skin_realsub) <- "RNA"

skin_realsub <- RunUMAP(skin_realsub, dims = 1:30, reduction.name = "umap.rna")
skin_realsub <- FindNeighbors(skin_realsub, dims = 1:30)
#skin_sub <- FindClusters(skin_sub, resolution = 0.5, algorithm = 3)

In [ ]:
DefaultAssay(skin_realsub) <- 'ATAC'

skin_realsub <- RunUMAP(skin_realsub, reduction = 'lsi', dims = 2:30, reduction.name = 'umap.atac')
skin_realsub <- FindNeighbors(skin_realsub, reduction = 'lsi', dims = 2:30)

In [ ]:
skin_realsub <- FindMultiModalNeighbors(skin_realsub, reduction.list = list("pca", "lsi"), dims.list = list(1:30, 2:30))
skin_realsub <- RunUMAP(skin_realsub, nn.name = "weighted.nn", reduction.name = "wnn.umap", reduction.key = "wnnUMAP_")

In [ ]:
skin_realsub <- FindClusters(skin_realsub, graph.name = "wsnn", resolution = 0.7)

In [ ]:
skin_realsub$celltype_reanno = 'NA'
skin_realsub$celltype_reanno[skin_realsub$seurat_clusters==0 | skin_realsub$seurat_clusters==3 | 
                             skin_realsub$seurat_clusters==8 | skin_realsub$seurat_clusters==9 | skin_realsub$seurat_clusters==7] <- 'TAC'
skin_realsub$celltype_reanno[skin_realsub$seurat_clusters==2] <- 'TAC'
skin_realsub$celltype_reanno[skin_realsub$seurat_clusters==4] <- 'IRS'
skin_realsub$celltype_reanno[skin_realsub$seurat_clusters==1] <- 'Hair Shaft-Cuticle/Cortex'
skin_realsub$celltype_reanno[skin_realsub$seurat_clusters==5] <- 'Medulla'
skin_realsub$celltype_reanno[skin_realsub$seurat_clusters==6] <- 'IRS'

In [ ]:
skin_realsub$lineage = 0
skin_realsub$lineage[skin_realsub$seurat_clusters==4 | skin_realsub$seurat_clusters==6 | 
                             skin_realsub$seurat_clusters==7 | skin_realsub$seurat_clusters==8] <- 1
skin_realsub$lineage[skin_realsub$seurat_clusters==2] <- 2
skin_realsub$lineage[skin_realsub$seurat_clusters==3] <- 3
skin_realsub$lineage[skin_realsub$seurat_clusters==9 | skin_realsub$seurat_clusters==1] <- 4
skin_realsub$lineage[skin_realsub$seurat_clusters==5] <- 5

In [ ]:
saveRDS(skin_realsub, "skin_TAC-IRS-HS.rna.atac.seuratobj.rds")

# DE analysis

In [ ]:
skin_realsub <- readRDS(file = 'skin_TAC-IRS-HS.rna.atac.seuratobj.rds')
skin_realsub

In [ ]:
DefaultAssay(skin_realsub) <- 'RNA'
skin_realsub

In [ ]:
DefaultAssay(skin_realsub) <- 'RNA'
markers <- FindAllMarkers(skin_realsub, only.pos = TRUE)
markers <- markers[markers$p_val_adj<0.05,]

In [ ]:
write.table(markers, 'markers_trajectory.txt', quote = F, row.names = F, sep = '\t')

# Building trajectory

In [ ]:
library(monocle3)
library(SeuratWrappers)

In [ ]:
recreate.partition <- c(rep(1, ncol(skin_realsub)))
names(recreate.partition) <- colnames(skin_realsub)
recreate.partition <- as.factor(recreate.partition)
skin_realsub$monocle3_partitions <- recreate.partition

skin_realsub$monocle3_clusters <- skin_realsub$celltype_reanno

In [ ]:
DefaultAssay(skin_realsub) <- 'RNA'

In [ ]:
### Construct the basic cds object
cds_from_seurat <- as.cell_data_set(skin_realsub, reductions = 'wnn.umap', default.reduction = 'wnn.umap')
reducedDimNames(cds_from_seurat)[1] <- "UMAP"
names(cds_from_seurat@clusters) <- "UMAP"
cds_from_seurat@clusters@listData[["UMAP"]][["louvain_res"]] <- "NA"
cds_from_seurat

In [ ]:
cds_from_seurat <- learn_graph(cds_from_seurat, use_partition = TRUE)

In [ ]:
cds_from_seurat <- order_cells(cds_from_seurat, root_cells = 'R1.46.R2.30.R3.52.P1.55')

In [ ]:
options(repr.plot.width=7.2, repr.plot.height=6)
p <- plot_cells(
  cds = cds_from_seurat,
  color_cells_by = "pseudotime",
  show_trajectory_graph = TRUE
)
ggsave("/nfs/public/xixi/scRegulate/figures/fig5c.pdf", p, width = 7.2, height = 6)

In [ ]:
options(repr.plot.width=6.3, repr.plot.height=6)
plot_cells(
  cds = cds_from_seurat,
  color_cells_by = "seurat_clusters",
  show_trajectory_graph = TRUE
)

In [ ]:
skin_realsub$monocle3_pseudotime <- cds_from_seurat@principal_graph_aux@listData$UMAP$pseudotime

In [ ]:
saveRDS(skin_realsub, "skin_TAC-IRS-HS.rna.atac.seuratobj.rds")

# Aggregate cells (according to clusters)

In [ ]:
skin_realsub <- readRDS(file = 'skin_TAC-IRS-HS.rna.atac.seuratobj.rds')
skin_realsub

In [ ]:
library(DIRECTNET)

In [ ]:
estimateSizeFactorsForDenseMatrix <- function(counts, locfunc = median, round_exprs=TRUE, method="mean-geometric-mean-total"){

  CM <- counts
  if (round_exprs)
    CM <- round(CM)
  if (method == "weighted-median"){
    log_medians <- apply(CM, 1, function(cell_expr) {
      log(locfunc(cell_expr))
    })

    weights <- apply(CM, 1, function(cell_expr) {
      num_pos <- sum(cell_expr > 0)
      num_pos / length(cell_expr)
    })

    sfs <- apply( CM, 2, function(cnts) {
      norm_cnts <-  weights * (log(cnts) -  log_medians)
      norm_cnts <- norm_cnts[is.nan(norm_cnts) == FALSE]
      norm_cnts <- norm_cnts[is.finite(norm_cnts)]
      #print (head(norm_cnts))
      exp( mean(norm_cnts) )
    })
  }else if (method == "median-geometric-mean"){
    log_geo_means <- rowMeans(log(CM))

    sfs <- apply( CM, 2, function(cnts) {
      norm_cnts <- log(cnts) -  log_geo_means
      norm_cnts <- norm_cnts[is.nan(norm_cnts) == FALSE]
      norm_cnts <- norm_cnts[is.finite(norm_cnts)]
      #print (head(norm_cnts))
      exp( locfunc( norm_cnts ))
    })
  }else if(method == "median"){
    row_median <- apply(CM, 1, median)
    sfs <- apply(Matrix::t(Matrix::t(CM) - row_median), 2, median)
  }else if(method == 'mode'){
    sfs <- estimate_t(CM)
  }else if(method == 'geometric-mean-total') {
    cell_total <- apply(CM, 2, sum)
    sfs <- log(cell_total) / mean(log(cell_total))
  }else if(method == 'mean-geometric-mean-total') {
    cell_total <- apply(CM, 2, sum)
    sfs <- cell_total / exp(mean(log(cell_total)))
  }

  sfs[is.na(sfs)] <- 1
  sfs
}

In [ ]:
skin.aggregate <- Aggregate_data(skin_realsub, k_neigh = 30, size_factor_normalize = FALSE)

In [ ]:
rna <- skin.aggregate$rna[rowSums(skin.aggregate$rna)>0, ]
rna

In [ ]:
rna_new <- t(t(log(skin.aggregate$rna+1))/estimateSizeFactorsForDenseMatrix(skin.aggregate$rna))
rna_new <- rna_new[rowSums(rna_new)>0, ]
rna_new

In [ ]:
atac_new <- t(t(log(skin.aggregate$atac+1))/estimateSizeFactorsForDenseMatrix(skin.aggregate$atac))
atac_new <- atac_new[rowSums(atac_new)>0, ]
atac_new

In [ ]:
write.csv(rna_new, '../SHAREseq/rna.aggregate_30cells.csv')
write.csv(atac_new, '../SHAREseq/atac.aggregate_30cells.csv')

In [ ]:
sample <- skin.aggregate$cell_sample
sample <- cbind(sample, celltype = as.character(skin_realsub$lineage[skin.aggregate$cell_sample[, 30]]))
sample

In [ ]:
aggr_pseudotime <- c()
for (i in 1:nrow(sample)){
    aggr_pseudotime <- c(aggr_pseudotime, mean(skin_realsub$monocle3_pseudotime[as.numeric(sample[i, 1:(ncol(sample)-1)])]))
}
aggr_pseudotime

In [ ]:
sample <- cbind(sample, aggr_pseudotime = aggr_pseudotime)
sample

In [ ]:
write.csv(sample, '../SHAREseq/skin.aggregate.cellid&cluster&pseudotime_30cells.csv')